In [134]:
from flask import Flask, request, jsonify, render_template
import random
import requests
import tensorflow
import speech_recognition as sr
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import os

In [135]:
app = Flask(__name__, template_folder='template', static_folder='static')
model=load_model(r'C:\Users\user\Documents\Projects_Docs\SIH_FINAL\SER\Emotion_Emergency.h5')

In [136]:
@app.route('/')
def home():
    return render_template('index.html')

In [137]:
@app.route('/audio',methods=['POST'])
def audio():
    return render_template('audio.html')

In [138]:
@app.route('/predict_',methods=['POST'])
def predict_():

    # with open('upload\audio.wav','wb') as f:
    #     f.write(request.data)

    audio_file =r'C:\Users\user\Documents\Projects_Docs\SIH_FINAL\SER\upload\audio.wav'
  
    Features = pd.read_csv(r'C:\Users\user\Documents\Projects_Docs\SIH_FINAL\SER\features_Emergency.csv')
    Y = Features['labels'].values
    encoder = OneHotEncoder()
    Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
    
    # NOISE
    def noise(data):
        noise_amp = 0.035*np.random.uniform()*np.amax(data)
        data = data + noise_amp*np.random.normal(size=data.shape[0])
        return data
    # # STRETCH
    # def stretch(data, rate=0.8):
    #     return librosa.effects.time_stretch(data, rate)
    # # PITCH
    # def pitch(data, sampling_rate, pitch_factor=0.7):
    #     return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

    def feat_ext(data):
        mfcc = np.mean(librosa.feature.mfcc(y=data, sr=22050).T, axis=0)
        return mfcc

    def get_feat(path):
        data, sample_rate = librosa.load(path, duration=5, offset=0.6)
        # normal data
        res1 = feat_ext(data)
        result = np.array(res1)
        #data with noise
        noise_data = noise(data)
        res2 = feat_ext(noise_data)
        result = np.vstack((result, res2))
        return result

    feature = get_feat(audio_file)
    test =np.expand_dims(feature, axis=2)
    livepreds = model.predict(test)
    livepredictions = (encoder.inverse_transform((livepreds)))


    return render_template('audio.html', prediction_text='{}'.format(livepredictions[0]))


In [139]:
# @app.route('/results',methods=['POST'])
# def results():
#     data = request.get_json(force=True)
#     prediction = model.predict(data.values())
    
#     return jsonify(prediction)

In [140]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [19/Aug/2022 13:51:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2022 13:51:36] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:36] "GET /static/css/indexStyle.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:36] "GET /static/css/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:37] "GET /static/css/images/bgimg2.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "POST /audio HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "GET /static/js/recorder.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "GET /static/js/try.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "GET /static/js/recordhelper.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:51:41] "GET /static/css/images/logo.png HTTP/1.

1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [19/Aug/2022 13:51:51] "POST /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2022 13:54:04] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:54:04] "GET /static/css/indexStyle.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:54:04] "GET /static/css/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2022 13:54:05] "GET /static/css/images/bgimg2.png HTTP/1.1" 304 -
